<a href="https://colab.research.google.com/github/cantbelieveimshook/Baseline-Models/blob/main/ResNet_50_Pre_Trained_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# connects colab to your google drive
# skip if your dataset is not on google drive or you're not using colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install preprocess

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%load_ext tensorboard
import glob
import os
from os import listdir
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import shuffle
import pickle, datetime
import preprocess as pp
import cv2
from pathlib import Path
import tensorflow as tf

from tensorflow import keras
from keras.datasets import cifar10
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, MaxPool2D
from keras.layers.convolutional import ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from keras.utils import np_utils
from tensorflow.keras import optimizers
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator



import  PIL.Image

In [ ]:
data_path = 'put your dataset path here'
img_path= data_path


os.chdir(img_path) # changes the current working directory to the file path specified. This directory should be the directory of data you plan on using for the model'
print(os.path.abspath(os.getcwd()))

/content/drive/.shortcut-targets-by-id/1tigMWT0fiaXboH1G3YFC0f5RZOEUYf9L/Big Data REU 2022 Team 1/REU 2022 Research/Datasets/new-test-data_Seraj/fft-denoised


In [ ]:
# stops training if training accuracy exceeds 97%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.97):
      print("\nReached 97.0% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
batch_size = 32


# # this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rotation_range = 40,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        )

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(
        rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        './train',  # this is the target directory
        target_size=(256, 256),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        './validation',
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='binary')

# this is a similar generator, for validation data
test_generator = test_datagen.flow_from_directory(
        './test',
        target_size=(256, 256),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False)


Found 710 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
Found 236 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import RMSprop

# import pre-trained model
base_model = ResNet50(input_shape = (256,256,3), include_top = False, weights = 'imagenet')


x = layers.GlobalAveragePooling2D()(base_model.output)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(base_model.input, x)

model.compile(optimizer = "Adam", loss = 'binary_crossentropy', metrics = ['accuracy'])



94781440/94765736 [==============================] - 2s 0us/step


In [ ]:
callbacks = myCallback()

k = 100
h2 = model.fit(        
        train_generator,
        epochs = k,
        validation_data=validation_generator,
        callbacks = [callbacks],
        )

Epoch 1/100
23/23 [==============================] - 296s 12s/step - loss: 0.8888 - accuracy: 0.6423 - val_loss: 9275291.0000 - val_accuracy: 0.5000
Epoch 2/100
23/23 [==============================] - 29s 1s/step - loss: 0.6402 - accuracy: 0.6507 - val_loss: 213212.6719 - val_accuracy: 0.5000
Epoch 3/100
23/23 [==============================] - 29s 1s/step - loss: 0.6065 - accuracy: 0.6761 - val_loss: 32530.5352 - val_accuracy: 0.5000
Epoch 4/100
23/23 [==============================] - 29s 1s/step - loss: 0.5763 - accuracy: 0.6859 - val_loss: 4930.8101 - val_accuracy: 0.5000
Epoch 5/100
23/23 [==============================] - 29s 1s/step - loss: 0.5599 - accuracy: 0.7028 - val_loss: 270.3857 - val_accuracy: 0.5000
Epoch 6/100
23/23 [==============================] - 29s 1s/step - loss: 0.5708 - accuracy: 0.7070 - val_loss: 27.9029 - val_accuracy: 0.5000
Epoch 7/100
23/23 [==============================] - 30s 1s/step - loss: 0.5611 - accuracy: 0.6789 - val_loss: 18.3556 - val_accura

In [ ]:
# set k to be the number of epochs before the training stopped
k = 100
 
print("Average training accuracy: ",sum(h2.history['accuracy'])/k)
print("Average training loss: ",sum(h2.history['loss'])/k)
print("Average validation accuracy: ",sum(h2.history['val_accuracy'])/k)
print("Average validation loss: ",sum(h2.history['val_loss'])/k)

Average training accuracy:  0.6794084519147873
Average training loss:  0.2647756341099739
Average validation accuracy:  0.4429285708069801
Average validation loss:  95264.71096136034


In [ ]:
# plot training and validation accuracies
plt.plot(h2.history['accuracy'])
plt.plot(h2.history['val_accuracy'])
plt.title('Training and Validation Accuracies')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc = 'upper left')
plt.show()

In [ ]:
# plot training and validation losses
plt.plot(h2.history['loss'])
plt.plot(h2.history['val_loss'])
plt.title('Training and Validation Losses')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc = 'upper left')
plt.show()

In [ ]:
import sklearn.metrics as metrics

# get confusion matrix
pred = model.predict(validation_generator)

print("Confusion Matrix: \n")
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())
pred = np.round(pred)
confusion_matrix = metrics.confusion_matrix(y_true=true_classes, y_pred=pred)
confusion_matrix


Confusion Matrix: 



array([[11, 59],
       [10, 60]])

In [ ]:
# get test accuracy
_, acc = model.evaluate(test_generator)
print(acc)

8/8 [==============================] - 116s 16s/step - loss: 3.8731 - accuracy: 0.5508
0.5508474707603455
